In [2]:
import math
import random
import pygame
import sys
import numpy as np
import json
import os
import pandas as pd

# Simulating Boids Algorithm

Includes code on Boids Algorithm and saving that information over time.

In [39]:
# Size of canvas. These get updated to fill the whole screen.
width = 1000
height = 1000

numBoids = 100
visualRange = 75

boids = []
 
def initBoids():
    global boids
    boids = []
    for i in range(numBoids):
        boids.append({
            'x': random.random() * width,
            'y': random.random() * height,
            'dx': random.random() * 10 - 5,
            'dy': random.random() * 10 - 5,
            'history': [],
            # 'all_history': []
        })

    df = pd.DataFrame(boids) # Create a DataFrame For Boids
    df.drop(labels='history', axis=1, inplace=True) # Drop the history column
    df['Boids'] = [i for i in range(len(boids))] # Add column 'Boids' for boid number
    return df
def distance(boid1, boid2):
    return math.sqrt((boid1['x'] - boid2['x'])**2 + (boid1['y'] - boid2['y'])**2)

def nClosestBoids(boid, n):
    sorted_boids = sorted(boids, key=lambda other_boid: distance(boid, other_boid))
    return sorted_boids[1:n+1]

def sizeCanvas():
    global width, height
    size = (width, height)
    return pygame.display.set_mode(size)

def keepWithinBounds(boid):
    margin = 200
    turnFactor = 1

    if boid['x'] < margin:
        boid['dx'] += turnFactor
    if boid['x'] > width - margin:
        boid['dx'] -= turnFactor
    if boid['y'] < margin:
        boid['dy'] += turnFactor
    if boid['y'] > height - margin:
        boid['dy'] -= turnFactor

def flyTowardsCenter(boid):
    centeringFactor = 0.005

    centerX = 0
    centerY = 0
    numNeighbors = 0

    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
            centerX += otherBoid['x']
            centerY += otherBoid['y']
            numNeighbors += 1

    if numNeighbors:
        centerX /= numNeighbors
        centerY /= numNeighbors

        boid['dx'] += (centerX - boid['x']) * centeringFactor
        boid['dy'] += (centerY - boid['y']) * centeringFactor

def avoidOthers(boid):
    minDistance = 20
    avoidFactor = 0.05
    moveX = 0
    moveY = 0

    for otherBoid in boids:
        if otherBoid != boid:
            if distance(boid, otherBoid) < minDistance:
                moveX += boid['x'] - otherBoid['x']
                moveY += boid['y'] - otherBoid['y']

    boid['dx'] += moveX * avoidFactor
    boid['dy'] += moveY * avoidFactor

def matchVelocity(boid):
    matchingFactor = 0.05

    avgDX = 0
    avgDY = 0
    numNeighbors = 0

    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
            avgDX += otherBoid['dx']
            avgDY += otherBoid['dy']
            numNeighbors += 1

    if numNeighbors:
        avgDX /= numNeighbors
        avgDY /= numNeighbors

        boid['dx'] += (avgDX - boid['dx']) * matchingFactor
        boid['dy'] += (avgDY - boid['dy']) * matchingFactor

def limitSpeed(boid):
    speedLimit = 10

    speed = math.sqrt(boid['dx']**2 + boid['dy']**2)
    if speed > speedLimit:
        boid['dx'] = (boid['dx'] / speed) * speedLimit
        boid['dy'] = (boid['dy'] / speed) * speedLimit

def drawBoid(screen, boid):
    angle = math.atan2(boid['dy'], boid['dx'])
    boid_surface = pygame.Surface((30, 10), pygame.SRCALPHA)
    pygame.draw.polygon(boid_surface, (85, 140, 244), [
        (0, 0), (0, 10), (-15, 5)
    ])
    rotated_boid = pygame.transform.rotate(boid_surface, math.degrees(angle))
    rotated_rect = rotated_boid.get_rect(center=(boid['x'], boid['y']))
    screen.blit(rotated_boid, rotated_rect)

    if DRAW_TRAIL:
        for point in boid['history']:
            pygame.draw.circle(screen, (85, 140, 244, 102), (int(point[0]), int(point[1])), 1)

# Main animation loop
def animationLoop(animate=False):
    global boids
    for boid in boids:
        flyTowardsCenter(boid)
        avoidOthers(boid)
        matchVelocity(boid)
        limitSpeed(boid)
        keepWithinBounds(boid)

        boid['x'] += boid['dx']
        boid['y'] += boid['dy']
        boid['history'].append((boid['x'], boid['y']))
        boid['history'] = boid['history'][-50:]

    if animate:
        screen.fill((255, 255, 255))
        for boid in boids:
            drawBoid(screen, boid)

        pygame.display.flip()
        pygame.time.Clock().tick(60)

    df = pd.DataFrame(boids)
    df.drop(labels='history', axis=1, inplace=True)
    df['Boids'] = [i for i in range(len(boids))]
    
    return df

pygame.init()
screen = sizeCanvas()
DRAW_TRAIL = False

# initBoids()
# while True:
#     for event in pygame.event.get():
#         if event.type == pygame.QUIT:
#             pygame.quit()
#             sys.exit()
#     animationLoop()

## Running Boids Algorithm and Saving Information Into Pandas DataFrame

In [24]:
directory = '../data/myjson'
if not os.path.exists(directory):
    os.makedirs(directory)

In [40]:
import pandas as pd

num_sims = 10
num_time_steps = 1000

# Initialize final_df as an empty DataFrame
final_df = pd.DataFrame()

for n in range(num_sims):  # Run simulation num_sims times
    curr_df_list = []  # List to store DataFrames for current simulation
    curr_df = initBoids()  # Initialize boids with different positions and velocities
    curr_df['Simulation'] = [n] * len(curr_df)  # Set Simulation number
    curr_df['Timestep'] = [0] * len(curr_df)  # Set Timestep number to 0
    curr_df_list.append(curr_df)  # Append initial state to list

    for t in range(1, num_time_steps):  # Run animationLoop() for num_time_steps
        new_df = animationLoop()  # Updates boids to have new positions and velocities
        new_df['Simulation'] = [n] * len(new_df)  # Set Simulation number
        new_df['Timestep'] = [t] * len(new_df)  # Set Timestep number
        curr_df_list.append(new_df)  # Append updated state to list

    # Concatenate all DataFrames in the list once per simulation
    curr_df = pd.concat(curr_df_list, ignore_index=True)
    # Append the result of the current simulation to final_df
    final_df = pd.concat([final_df, curr_df], ignore_index=True)
final_df.reset_index()

In [42]:
final_df

,x,y,dx,dy,Boids,Simulation,Timestep
0,16.965684,307.122389,0.587605,-4.896190,0,0,0
1,686.809076,117.394859,2.394810,1.665589,1,0,0
2,465.946812,948.488950,-2.466515,-3.243389,2,0,0
3,432.335856,888.928757,3.258690,4.327373,3,0,0
4,385.435550,386.359073,-2.969556,-0.164881,4,0,0
...,...,...,...,...,...,...,...
999995,493.410297,718.114052,1.771291,-8.071311,95,9,999
999996,424.605820,750.497793,0.771057,-9.970229,96,9,999
999997,382.804529,691.214181,0.738960,-9.972660,97,9,999
999998,265.363650,737.026778,0.126421,-9.818494,98,9,999


## Converting Simulation DataFrame to CSV

In [27]:
path_to_save = '../data/simulation.csv'
final_df.to_csv(path_to_save, index=False)

## Save Edges From Each Timestep/Simulation

In [43]:
path_to_save = '../data/simulation.csv'
final_df = pd.read_csv(path_to_save)

### Compute The Edges Per Timestep Per Simulation

In [46]:
from scipy.spatial.distance import pdist, squareform
# Distance threshold

# Function to calculate pairwise distances and return edges
def get_edges(df, threshold):
    
    distances = squareform(pdist(df[['x', 'y']])) # Calculate pairwise distances
    
    close_pairs = distances < threshold # Identify pairs within the threshold distance
    
    # Extract indices of close pairs
    edges = [(i, j) for i in range(len(distances)) for j in range(i+1, len(distances)) if close_pairs[i, j]]
    
    # Create DataFrame for edges
    edges_df = pd.DataFrame(edges, columns=['Boid_i', 'Boid_j'])
    
    return edges_df


edges_dfs = []  # List to collect DataFrames
final_df_groupby = final_df.groupby(['Timestep', 'Simulation'])
for key, item in final_df_groupby:
    edges_df = get_edges(item, visualRange)  # Pass the group directly
    edges_df['Timestep'] = key[0]
    edges_df['Simulation'] = key[1]
    edges_dfs.append(edges_df)

# Concatenate all DataFrames at once, if edges_dfs is not empty
if edges_dfs:
    final_edges_df = pd.concat(edges_dfs, ignore_index=True)
    final_edges_df.reset_index(drop=True, inplace=True)  # Reset index once, outside the loop
else:
    final_edges_df = pd.DataFrame()  # Initialize to an empty DataFrame if no edges were found

final_edges_df

,Boid_i,Boid_j,Timestep,Simulation
0,0,81,0,0
1,1,50,0,0
2,2,26,0,0
3,2,57,0,0
4,3,7,0,0
...,...,...,...,...
6841124,91,95,999,9
6841125,92,94,999,9
6841126,96,97,999,9
6841127,96,99,999,9


### Save final_edges_df As A CSV

In [47]:
path_to_save = '../data/simulation_edges.csv'
final_edges_df.to_csv(path_to_save, index=False)